In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import win32com.client
import getpass
import datetime
import pywintypes
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

# Function

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

# excel to df
def getDataToDF(xlws,start,end,first = False):
    last_col = xlws.UsedRange.Columns.Count
    header = xlws.Range(xlws.Cells(1, 1), xlws.Cells(1, last_col)).Value
    content = xlws.Range(xlws.Cells(start, 1), xlws.Cells(end, last_col)).Value
    
    if first:
        data = list(content[1:])
    else:
        data = list(content[0:])
        
    for x in range(0,len(data)):
        data[x] = list(data[x])
        for y in range(0,len(data[x])):
            if isinstance(data[x][y], pywintypes.TimeType):
                temp = str(data[x][y]).rstrip("+00:00").strip()
                if temp[:2] == '20' and int(temp[:4]) >= int(now.year):
                    temp = '19' + temp[2:]
                data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")
    df = pd.DataFrame(data, columns=header[0])
    df.fillna(value=pd.np.nan, inplace=True)
    return df

def processCol(df):                
    ####need to compute timeOS
            
    #for those dropdown variables in dictionary
    toDoDict = {}
    toDoDict['Gender'] = [('1','2','4','5','6','9997','9998','9999'),
                          ['Male','Female','Transsexual, NOS','Transsexual, natal male','Transsexual, natal female','NA',
                           'Others (intersex, disorders of sexual development/DSD)','Unknown']]
    toDoDict['c_tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                            ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                             'T4a','T4b','T4c','T4d']]
    toDoDict['cNstage'] = [('1','2','3','4','9','10','31','32','41','42','43'),
                           ['N0','N1','N2','N3','Nx','unknown','N2a','N2b','N3a','N3b','N3c']]
    toDoDict['cMstage'] = [('0','1','2','3','4','5','6','9997','9999'),
                           ['MX','M0','M1','M1a','M1b','M1c','M2','NA','Unknown']]     
    toDoDict['c_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']] 
    toDoDict['tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                          ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                           'T4a','T4b','T4c','T4d']] 
    toDoDict['nstage'] = [('1','2','3','4','5','6','9','10','21','22','23','31','32','41','42','43'),
                          ['N0','N1','N2','N3','N1mic','N0 (i+)','Nx','unknown','N1a','N1b','N1c',\
                           'N2a','N2b','N3a','N3b','N3c']]
    toDoDict['Mstage'] = [('1','2','3','9'),('M0','M1','Mx','unknown')] 
    toDoDict['p_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']]
    toDoDict['diff'] = [('0','1','2','3','9'),['grade 0','grade 1','grade 2','grade 3','unknown']] 
    toDoDict['TNM_Stage'] = [('0','1','2','3','4','5','6','7','9','10','11','12','13','31'),
                             ['DCIS/LCIS non-invasive','stage 1','stage 2A','stage 2B',\
                              'stage 3A','stage 3B','stage 3C','stage 4','Unknown','Stage 0',\
                              'stage 1A','stage 1B','Stage 3','stage 2']]
    toDoDict['ProgStage_AJCC8'] = [('0','4','11','12','21','22','31','32','33'),
                                   ['Stage 0','Stage IV','Stage IA','Stage IB','Stage IIA',\
                                   'Stage IIB','Stage IIIA','Stage IIIB','Stage IIIC']] 
    toDoDict['ER'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']] 
    toDoDict['PR'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['cerbB2'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['Her2'] = [('1','2','3','4','9'),['Positive','Negative','Not Done','Equivocal','Unknown']]
    toDoDict['cause_of_death'] = [('1','2','9'),['breast cancer related','N','unknown']]
    toDoDict['Count_as_DFS'] = [('0','1'),['N' ,'Disease recurrence ANY']]
    toDoDict['Count_as_OS'] = [('0','1'),['N' ,'Dead']]
    
    tmp = df.copy(deep=True)
    for k,v in toDoDict.items():
#         print("Shape before drop at {}: {}, {}".format(k,df.shape, tmp.shape))
        
        v[1] = [x.lower() for x in v[1]]
        df[k].replace(to_replace =v[0],\
                      value = v[1],\
                      inplace = True)
        df[k] = df[k].str.lower()
        
        # allow null values
        v[1].append(pd.np.nan)
        
        # drop abnormal values   
        df.drop(df.loc[~df[k].isin(v[1])].index, inplace=True)
        
#         print("Shape after drop at {}: {}, {}".format(k, df.shape, tmp.shape))

    DateList =  {"death_age":('death','dob')}
    for k,v in DateList.items():
        df[v[0]] = pd.to_datetime(df[v[0]])
        df[v[1]] = pd.to_datetime(df[v[1]])
        df[k] = (df[v[0]] - df[v[1]]).dt.days
        df[k] = df[k].floordiv(365, fill_value = pd.np.nan) 
        df = df.drop(columns=v[0])
    return df, tmp

def dropColCDM(df, listToDrop):
    df = df.drop(columns=listToDrop)
    return df

def oneHotEncode(df, listToDo):
    for x in listToDo:
        df = pd.concat([df,pd.get_dummies(df[x], prefix=x, dummy_na=True)],axis=1)
        df.drop([x],axis=1, inplace=True)        
    return df
        
def outToPickle(df,filename):
    with open(filename, 'wb') as f:
        pickle.dump(df, f)

# Main Process

In [ ]:
FileToCheck = 'C:\\SMU_v2\\output.pkl'

if os.path.exists(FileToCheck):
    with open(FileToCheck, 'rb') as f:
        CDM = pickle.load(f)
else:
    # primary set up
    xlApp = win32com.client.Dispatch("Excel.Application")
    xlApp.Interactive = False
    xlApp.Visible = False

    # cwd = os.getcwd()
    cwd = "C:"
    path = str(cwd + "\\SMU_v2\\")
    files = os.listdir(path)

    # Put files into dataframe dict
    df_list = {}

    #require user input for password
    pwd = getpass.getpass('Enter file password: ')

    # Pick out 'xlsx' files:
    files_xls = [f for f in files if f[-4:] == 'xlsx']

    now = datetime.datetime.now()

    for f in files_xls:
        xlwb = xlApp.Workbooks.Open(path+f, False, True, None, pwd)
        xlws = xlwb.Worksheets(1) 
        last_row = xlws.UsedRange.Rows.Count

        if "Clinical" in f:
                CDM = getDataToDF(xlws,1,last_row )
                listToKeep = ['NRIC','dob','Gender','dx_date','c_tstage','cNstage','cMstage','c_Staging','tstage',\
                              'nstage','Mstage','p_Staging','diff','TNM_Stage','ProgStage_AJCC8','ER','PR',\
                             'cerbB2','Her2','size_precise','nodespos','cause_of_death','Date_for_DFS','Date_for_OS',\
                             'Count_as_DFS','Count_as_OS','Age_@_Dx','death']
                listToDrop = [i for i in CDM.columns if i not in listToKeep]

                CDM = dropColCDM(CDM,listToDrop)
                CDM,tmp = processCol(CDM)
                
                listToDoforOHE = ['Gender','c_tstage','cNstage','cMstage','c_Staging','tstage','nstage','Mstage',\
                                  'p_Staging','diff','TNM_Stage','ProgStage_AJCC8','ER','PR','cerbB2','Her2',\
                                  'cause_of_death','Count_as_DFS','Count_as_OS']
                
                CDM = oneHotEncode(CDM,listToDoforOHE)
                outToPickle(CDM,FileToCheck )  
        else:
            # create name for bill dataframe
            billName = 'bill{}'.format(x)
    # #         rows to loop each time
    #         rowToDo = 
    #         cvsName = ""
    # #         put individual bill df to df_list
    #         df_list[billName] = getDataToDF(xlws,rowToDo)
            x+=1

        #reset variables
        xlws = None
        xlwb.Close(False)
        xlwb = None

    #remove buffer and reset system settings
    xlApp.Interactive = True
    xlApp.Visible = True
    xlApp.Quit()
    xlApp = None

In [ ]:
CDM.head(2)

In [ ]:
# CDM[['death_age']].plot(kind='hist',bins=[0,20,40,60,80,100],rwidth=0.8)
# plt.show()
CDM['death_age']

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


X = CDM.iloc[:,0:20]  #independent columns
y = CDM.iloc[:,-1]    #target column i.e price range

#apply SelectKBest class to extract top 100 best features
bestfeatures = SelectKBest(score_func=chi2, k=100)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(100,'Score'))  #print 10 best features

# MY

In [ ]:
#for bills
test_patient = "4b6ac0036ae2a4e8e6c3"
# 'Bills Data_10-12k (MASKED)v2.xlsx', 'Bills Data_12-14k (MASKED)v2.xlsx', 'Bills Data_14-16k (MASKED)v2.xlsx', 'Bills Data_16-18k (MASKED)v2.xlsx', 'Bills Data_18-20k (MASKED)v2.xlsx', 'Bills Data_1st 2k (MASKED)v2.xlsx', 'Bills Data_2-4k (MASKED)v2.xlsx', 'Bills Data_20-22k (MASKED)v2.xlsx', 'Bills Data_22-24k (MASKED)v2.xlsx', 'Bills Data_24-26k (MASKED)v2.xlsx', 'Bills Data_4-6k (MASKED)v2.xlsx', 'Bills Data_6-8k (MASKED)v2.xlsx', 'Bills Data_8-10k (MASKED)v2.xlsx', 'Bills Data_last 1k (MASKED)v2.xlsx', 'Breast Database Dictionary SMU_v2.xlsx', 'Clinical Data_Masked_v2.xlsx', 'Mapping for service code.xlsx'
bills = df_list['Bills Data_last 1k (MASKED)v2.xlsx']
patients = df_list['Clinical Data_Masked_v2.xlsx']

In [ ]:
test_patient_bills = bills[bills["Patient.ID"]==test_patient]
est_patient_dx = patients[patients["NRIC"]==test_patient]['dx_date']

In [ ]:
earliest_date = test_patient_dx - pd.tseries.offsets.MonthOffset(6)
earliest_date = earliest_date.values[0]

In [ ]:
latest_date =min(pd.to_datetime('now').to_datetime64(), (test_patient_dx + pd.tseries.offsets.MonthOffset(120)).values[0])

In [ ]:
#only looking at bills from NCC
test_patient_bills = test_patient_bills[test_patient_bills['Institution.Code'] == "NCC"]

In [ ]:
#remove all bills not in the range of dates from 6 months before diagnosis and 10 years after diagnosis
test_patient_bills["Service.Date.From.Date"]

In [ ]:
resampled = test_patient_bills[["Net.Payable","Service.Date.From.Date"]].resample("3M",on = "Service.Date.From.Date").sum()

In [ ]:
resampled.mean()

In [ ]:
test_patient_dx

In [ ]:
test_patient_bills[test_patient_bills["Admit.Date"] - test_patient_dx]